In [13]:
import os
from typing import List
from pathlib import Path
import pandas as pd
ROOT_DIR : str = Path(os.getcwd())
DATA_DIR : str = ROOT_DIR/"data"
SIH_DIR : str = DATA_DIR/"sih"
IBGE_DATA : str = DATA_DIR/"tabelas_ibge"

In [14]:
def create_folder(path:str)->None:
    if not os.path.exists(path): 
        os.makedirs(path)
def get_subdirectories(parent_directory:str)->List[str]:
    subdirectories = []
    for item in os.listdir(parent_directory):
        item_path = os.path.join(parent_directory, item)
        subdirectories.append(item_path)
    return subdirectories

In [15]:
def load_files()->pd.DataFrame:
    list_of_files : List[pd.DataFrame] = []
    list_of_dirs = get_subdirectories(SIH_DIR)
    for item in list_of_dirs:
        list_of_files.append(pd.read_parquet(item))
    return pd.concat(list_of_files)

In [16]:
df_sih: pd.DataFrame = load_files()

In [17]:
df_sih.columns

Index(['UF_ZI', 'ANO_CMPT', 'MES_CMPT', 'ESPEC', 'CGC_HOSP', 'N_AIH', 'IDENT',
       'CEP', 'MUNIC_RES', 'NASC',
       ...
       'DIAGSEC9', 'TPDISEC1', 'TPDISEC2', 'TPDISEC3', 'TPDISEC4', 'TPDISEC5',
       'TPDISEC6', 'TPDISEC7', 'TPDISEC8', 'TPDISEC9'],
      dtype='object', length=113)

In [18]:
# Vai ter que carregar uma tabela com a referência SID-10
# Vai ter que carregar uma tabela com o código ibge dos municípios
colunas_selecionadas : List[str] = [
    'N_AIH',
    'UF_ZI',
    'ANO_CMPT',
    'MES_CMPT',
    'DIAG_PRINC',
    'IDENT',
    'CEP',
    'MUNIC_RES',
    'VAL_SH',
    'NASC',
    'SEXO',
    'QT_DIARIAS',
    'VAL_TOT',
    'DIAS_PERM',
    'CID_NOTIF'

]

In [19]:
# Seleciona colunas
df_sih_sel = df_sih[colunas_selecionadas]
df_sih_sel.head()

,N_AIH,UF_ZI,ANO_CMPT,MES_CMPT,DIAG_PRINC,IDENT,CEP,MUNIC_RES,VAL_SH,NASC,SEXO,QT_DIARIAS,VAL_TOT,DIAS_PERM,CID_NOTIF
0,4315103009712,430000,2015,07,O908,1,95625000,431033,799.18,19830427,3,17,848.68,17,
1,4315103011010,430000,2015,07,O829,1,96270000,431250,411.68,19820828,3,2,678.11,2,
2,4315103012220,430000,2015,07,O459,1,95520000,431350,419.68,19850307,3,3,696.11,3,
3,4315103013749,430000,2015,07,O809,1,95520000,431350,283.60,19981010,3,2,524.60,2,
4,4315103019656,430000,2015,07,Z302,1,95560000,432150,411.68,19871006,3,2,688.11,2,Z302


In [20]:
# Cria tabela time_table - agrupando por ano e mês
def create_time_table(df:pd.DataFrame)->pd.DataFrame:
    selected_columns : List[str] = ['ANO_CMPT','MES_CMPT']
    df_time_table :pd.DataFrame = df[selected_columns]
    df_time_table = df_time_table.drop_duplicates(subset=selected_columns)
    return df_time_table.sort_values(by=selected_columns)
def create_time_table_key(df:pd.DataFrame)->pd.DataFrame:
    df['TIME_KEY'] = df['ANO_CMPT'] + '|' + df['MES_CMPT']
    return df
df_time_table = create_time_table(df=df_sih_sel)
df_time_table = create_time_table_key(df_time_table)
df_time_table.head()

,ANO_CMPT,MES_CMPT,TIME_KEY
0,2015,01,2015|01
0,2015,02,2015|02
0,2015,03,2015|03
0,2015,04,2015|04
0,2015,05,2015|05


In [21]:
def create_paciente_table(df:pd.DataFrame)->pd.DataFrame:
    df_paciente_table = df[['N_AIH','NASC','SEXO']]
    df_paciente_table = df_paciente_table.drop_duplicates()
    df_paciente_table = df_paciente_table.rename(columns={'N_AIH':'ID_PACIENTE'})
    return df_paciente_table
def convert_date_to_datetime(df:pd.DataFrame)->pd.DataFrame:
    df['NASC'] = pd.to_datetime(df['NASC'], format='%Y%m%d')
    df['NASC'] = df['NASC'].dt.strftime('%d/%m/%Y')
    return df
df_paciente_table = create_paciente_table(df_sih_sel)
df_paciente_table = convert_date_to_datetime(df=df_paciente_table)
df_paciente_table.head()

,ID_PACIENTE,NASC,SEXO
0,4315103009712,27/04/1983,3
1,4315103011010,28/08/1982,3
2,4315103012220,07/03/1985,3
3,4315103013749,10/10/1998,3
4,4315103019656,06/10/1987,3


In [22]:
def create_municipios_table()->pd.DataFrame:
    df_municipios :pd.DataFrame = pd.read_csv(filepath_or_buffer=IBGE_DATA/'municipios.csv', sep=',')
    return df_municipios.rename(columns={'COD':'COD_IBGE', 'COD UF':'COD_UF'})
def create_estados_table()->pd.DataFrame:
    df_estados:pd.DataFrame = pd.read_csv(filepath_or_buffer=IBGE_DATA/'estados.csv',sep=',')
    return df_estados.rename(columns={'COD':'COD_UF'})
df_municipios:pd.DataFrame = create_municipios_table()
df_estados:pd.DataFrame = create_estados_table()

In [24]:
# Cria time_key
df_sih_sel['TIME_KEY'] = df_sih_sel['ANO_CMPT'] + '|' + df_sih_sel['MES_CMPT']

Index(['N_AIH', 'UF_ZI', 'ANO_CMPT', 'MES_CMPT', 'DIAG_PRINC', 'IDENT', 'CEP',
       'MUNIC_RES', 'VAL_SH', 'NASC', 'SEXO', 'QT_DIARIAS', 'VAL_TOT',
       'DIAS_PERM', 'CID_NOTIF'],
      dtype='object')

In [25]:
df_sih_sel['CID_NOTIF'].unique()

array(['    ', 'Z302', 'O149', 'O450', 'O150', 'O992', 'O990'],
      dtype=object)